<a href="https://colab.research.google.com/github/naokityokoyama/HDC/blob/main/HDC_ngram.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch-hd binhd unidecode num2words torchmetrics -q

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 361.0/361.0 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.8/235.8 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.5/163.5 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.9/981.9 kB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 62.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 34.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 34.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.1 MB/s eta 0:00:0

In [2]:
import zipfile
from unidecode import unidecode
import string
from num2words import num2words
import re
import numpy as np
import pandas as pd
from typing import Union, Literal
from tqdm import tqdm

import torch
from torch.utils.data import DataLoader
import torch.nn as nn
import torchhd
from torchhd import embeddings

from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from binhd.embeddings import ScatterCode
from binhd.datasets import BaseDataset
from binhd.classifiers import BinHD, NeuralHD
from torchmetrics import Accuracy


import warnings
warnings.filterwarnings("ignore")

In [3]:
path_origin = '/content/drive/MyDrive/uff/fake.zip'
path_destino = '/content/'
with zipfile.ZipFile(path_origin, "r") as zip_ref:
    zip_ref.extractall(path_destino)

In [4]:
#build dataset

df_fake = pd.read_csv('/content/fakes.csv')[['text']]
df_fake['target'] = 1
df_true = pd.read_csv('/content/true.csv')[['text']]
df_true['target'] = 0
df = pd.concat([df_fake, df_true]).reset_index(drop=True)

In [5]:
# Definir o tamanho da amostra
sample_size = 1000

# Criar uma amostra balanceada
df = df.groupby("target", group_keys=False).apply(lambda x: resample(x, n_samples=sample_size // df["target"].nunique(), random_state=42))
df = df.reset_index(drop=True)


In [6]:
df.head(3)

,text,target
0,"Entre 2007 e 2016, cerca de 553 mil pessoas mo...",0
1,As propinas descem pela primeira vez em 15 anos?,0
2,"O rendimento ""per capita"" dos portugueses é o ...",0


Clean

In [7]:
def n2w(texto:str)->str:
  padrao = r"\d+"
  numeros = re.findall(padrao, texto)
  for numero in numeros:
    extenso = num2words(numero, lang='pt')
    texto = texto.replace(numero, extenso)
  return texto

In [8]:
for repet in tqdm(range(2)):  #bug para rodar 2x
  df['text'] = df['text'].str.lower()
  df['text'] = df['text'].str.replace(f"[{string.punctuation}]", "", regex=True)
  df['text'] = df['text'].apply(lambda x: ' '.join(x.split()))
  df['text'] = df['text'].str.replace('"', '').str.replace('\\', '')
  df['text'] = df['text'].apply(n2w)
  df['text'] = df['text'].apply(unidecode)

100%|██████████| 2/2 [00:00<00:00, 26.75it/s]


In [9]:
df['text'][8]

'passamos para mais de trezentos trezentos e vinte cursos de medicina atualmente'

In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using {} device".format(device))

MAX_INPUT_SIZE = 128
PADDING_IDX = 0

ASCII_A = ord("a")
ASCII_Z = ord("z")
ASCII_SPACE = ord(" ")
NUM_TOKENS = ASCII_Z - ASCII_A + 3  # a through z plus space and padding
print (ASCII_A, '--', ASCII_Z, '--', ASCII_SPACE, '--', NUM_TOKENS)

def char2int(char: str) -> int:
    """Map a character to its integer identifier"""
    ascii_index = ord(char)

    if ascii_index == ASCII_SPACE:
        # Remap the space character to come after "z"
        return ASCII_Z - ASCII_A + 1

    return ascii_index - ASCII_A


def transform(x: str) -> torch.Tensor:
    char_ids = x[:MAX_INPUT_SIZE]
    char_ids = [char2int(char) + 1 for char in char_ids.lower()]

    if len(char_ids) < MAX_INPUT_SIZE:
        char_ids += [PADDING_IDX] * (MAX_INPUT_SIZE - len(char_ids))

    return torch.tensor(char_ids, dtype=torch.long)

Using cpu device
97 -- 122 -- 32 -- 28


In [11]:
#create X and y
lst = []
print ('size dataset',df.shape[0] )
for i in range(df.shape[0]):
  lst.append (np.array(transform(df['text'][i])))

X = np.array(lst)
y = list(df['target'])


size dataset 1000


In [62]:
X

array([[ 5, 14, 20, ..., 27, 16,  1],
       [ 1, 19, 27, ...,  0,  0,  0],
       [15, 27, 18, ...,  0,  0,  0],
       ...,
       [16, 18, 15, ...,  0,  0,  0],
       [21, 18,  7, ...,  0,  0,  0],
       [20, 18, 21, ...,  0,  0,  0]])

In [14]:
DIMENSION = 1000
NUM_LEVELS = 100 #usado no record encoding
BATCH_SIZE = 100
CLASSES = df['target'].nunique()

min_val = float(X.min())
max_val = float(X.max())

SIZE = X.shape[1]
print('val_min :', min_val, 'val_max :', max_val)

accuracy = Accuracy("multiclass", num_classes=CLASSES)
model_BindHD = BinHD(DIMENSION, CLASSES)
model_NeuralHD = NeuralHD(n_dimensions=128, n_classes=CLASSES, n_features=1000)


val_min : 0.0 val_max : 27.0


In [15]:
class NgramEncoder(nn.Module):
    def __init__(self, out_features, size):
        super(NgramEncoder, self).__init__()
        self.symbol = embeddings.Random(size, out_features, padding_idx=PADDING_IDX)
        #print(self.symbol)
    def forward(self, x):
        #print (x, x.shape)
        symbols = self.symbol(x)
        sample_hv = torchhd.ngrams(symbols, n=4)  #colocar no construtor
        return torchhd.normalize(sample_hv)

encode = NgramEncoder(DIMENSION, NUM_TOKENS)
encode = encode.to(device)

In [ ]:

# with torch.no_grad():
#     samples = torch.tensor(X).to(device)
#     labels = torch.tensor(y).squeeze().to(device)

#     X_hd = encode(samples)
#     X_hd = X_hd.clip(0,1)
#     X_Bind = X_hd.to(torch.int8)
# X_train_, X_test_1, y_train, y_test = train_test_split(X_Bind, labels, test_size=0.30, random_state = 42)
# X_train, X_test, y_train, y_test = train_test_split(X_hd, labels, test_size=0.30, random_state = 42)



In [ ]:
# model_BindHD = BinHD(DIMENSION, CLASSES)
# with torch.no_grad():
#     model_BindHD.fit(X_train_bind,y_train_bind)
#     predictions = model_BindHD.predict(X_test_bind)
#     acc = accuracy_score(predictions, y_test_bind)
#     print("BinHD: Accuracy = ", acc)

BinHD: Accuracy =  0.5033333333333333


In [ ]:
# model_NeuralHD = NeuralHD(n_dimensions=128, n_classes=2, n_features=1000)
# with torch.no_grad():
#     model_NeuralHD.fit(X_train_Neural,y_train_Neural)
#     predictions = model_NeuralHD.predict(X_test_Neural)
#     acc = accuracy_score(predictions, y_test_Neural)
#     print("NeuralHD: Accuracy = ", acc)

In [ ]:
# Ativar
# class RecordEncoder(nn.Module):
#     def __init__(self, out_features, size, levels, low, high):
#         super(RecordEncoder, self).__init__()
#         self.position = embeddings.Random(size, out_features, vsa="BSC", dtype=torch.uint8)
#         self.value = ScatterCode(levels, out_features, low = low, high = high)

#     def forward(self, x):
#         sample_hv = torchhd.bind(self.position.weight, self.value(x))
#         sample_hv = torchhd.multiset(sample_hv)
#         return sample_hv

# record_encode = RecordEncoder(DIMENSION, SIZE, NUM_LEVELS, min_val, max_val)
# record_encode = record_encode.to(device)

# with torch.no_grad():
#     samples = torch.tensor(X).to(device)
#     labels = torch.tensor(y).squeeze().to(device)

#     X_hv = record_encode(samples)

# X_train, X_test, y_train, y_test = train_test_split(X_hv, labels, test_size=0.3, random_state = 0)

# model = BinHD(DIMENSION, CLASSES)

# with torch.no_grad():
    # model.fit(X_train,y_train)
    # predictions = model.predict(X_test)
    # acc = accuracy_score(predictions, y_test)
    # print("BinHD: Accuracy = ", acc)

Batch

In [16]:
class train_hdc:
  def __init__(self, X: Union[float, int],
               y: Union[float, int],
               random:int = 42,
               test_size:float = 0.30,
               dimension:int=1000,
               batch_size:int=1000,
               classes:int=2,
               features:int=128):
    self.X = X
    self.y = y
    self.random = random
    self.test_size = test_size
    self.dimension = dimension
    self.batch_size = batch_size
    self.classes = classes
    self.features = features

  def prepare_data(self)->str:
    X = self.X
    y = self.y
    test_size = self.test_size
    random = self.random
    X_train_bind, X_test_bind, y_train_bind, y_test_bind = train_test_split(X, y, test_size=test_size, random_state =random)

    train_dataset = BaseDataset(X_train_bind, y_train_bind)
    test_dataset = BaseDataset(X_test_bind, y_test_bind)

    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE)
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)
    return train_loader, test_loader

  def train(self, model_name:Literal['bindhd', 'neuralhd'])-> str:
    #models
    DIMENSION = self.dimension
    CLASSES = self.classes
    BATCH_SIZE = self.batch_size
    FEATURES = self.features

    model_BindHD = BinHD(DIMENSION, CLASSES)
    model_NeuralHD = NeuralHD(n_dimensions=FEATURES, n_classes=CLASSES, n_features=DIMENSION)
    if model_name == 'bindhd':
      model = model_BindHD
    elif model_name == 'neuralhd':
      model = model_NeuralHD
    else:
      raise ValueError("Invalid model name provided. Choose 'bindhd' or 'neuralhd'.")

    #train BINHD
    data_train = self.prepare_data()[0]

    with torch.no_grad():
      for x_ , y_ in tqdm(data_train) :

        samples = torch.tensor(x_).to(device)
        labels = torch.tensor(y_).squeeze().to(device)
        if model_name  == 'bindhd':
          X_Bind = encode(samples)
          X_Bind = X_Bind.clip(0,1) #
          X_Bind = X_Bind.to(torch.int8)

          model_BindHD.fit(X_Bind,labels)
          trained_model = model_BindHD
        else:
          X_neural = encode(samples)
          model_NeuralHD.fit(X_neural,labels)
          trained_model = model_NeuralHD

      return trained_model


  def test(self, model:str)->str:
    data_test = self.prepare_data()[1]
    model_name = model.__class__.__name__

    if model_name == 'BinHD':
      model_BinHD = model
      print ('model BinHD')
    elif model_name == 'NeuralHD':
      print ('model NeuralHD')
      model_NeuralHD = model
    else:
      raise ValueError("Invalid model name provided. Choose 'bindhd' or 'neuralhd'.")
    with torch.no_grad():
      for x_ , y_ in data_test:

        samples = torch.tensor(x_).to(device)
        labels = torch.tensor(y_).squeeze().to(device)

        if model_name  == 'BinHD':
          X_Bind = encode(samples)
          #X_Bind = X_Bind.clip(0,1)
          X_Bind = X_Bind.to(torch.int8)
          predictions = model_BinHD.predict(X_Bind)
          accuracy.update(predictions, labels)

        elif model_name  == 'NeuralHD':
          X_Neural = encode(samples)
          predictions = model_NeuralHD.predict(X_Neural)
          accuracy.update(predictions, labels)

        else:
          raise ValueError("Invalid model name provided. Choose 'bindhd' or 'neuralhd'.")

    acc = accuracy.compute().item()
    print(f"{model_name}: Accuracy = ", acc)

In [17]:
hdc = train_hdc(X, y)

In [18]:
#train
modelo_bindhd = hdc.train(model_name='bindhd')
modelo_neuralhd= hdc.train(model_name='neuralhd')

100%|██████████| 7/7 [00:03<00:00,  1.80it/s]


In [19]:
modelo_bindhd

BinHD()

In [20]:
#test
hdc.test(model=modelo_bindhd)
hdc.test(model=modelo_neuralhd)

model BinHD
BinHD: Accuracy =  0.5166666507720947
model NeuralHD
NeuralHD: Accuracy =  0.5299999713897705


In [ ]:
# #train // test

# X_train_bind, X_test_bind, y_train_bind, y_test_bind = train_test_split(X, y, test_size=0.30, random_state = 42)
# train_dataset = BaseDataset(X_train_bind, y_train_bind)
# test_dataset = BaseDataset(X_test_bind, y_test_bind)

# train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE)
# test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)